# Computer Science Project

In [2]:
import pandas as pd
import numpy as np

#### 1. Normalize the *loan_lenders* table. In the normalized table, each row must have one *loan_id* and one *lender*.

In [3]:
loans_lenders = pd.read_csv("Datasets/loans_lenders.csv")

Normalizzare la tabella significa che ogni cella deve contenere una sola informazione. In questo caso la colonna *lender* possiede diversi nomi, quindi è necessario duplicare il valore di *loan_id* tante volte quante sono i lender.

In [4]:
loans_lenders

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
1387427,678999,"michael43411218, carol5987, gooddogg1, chris41..."
1387428,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
1387429,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
1387430,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


Useremo il metodo *.explode* di pandas che richiede però una lista.

In [5]:
loans_lenders["lenders"] = loans_lenders.lenders.apply(lambda x: x.split(","))

In [6]:
loans_lenders = loans_lenders.explode("lenders").reset_index(drop = True)

In [7]:
loans_lenders

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


#### 2. For each loan, add a column *duration* corresponding to the number of days between the *disburse time* and the *planned expiration time*. If any of those two dates is missing, also the *duration* must be missing.

In [8]:
loans = pd.read_csv("Datasets/loans.csv")

Per calcolare la durata trasformiamo in formato *datetime* le colonne che ci interessano.

In [9]:
loans["disburse_time"] = pd.to_datetime(loans["disburse_time"])
loans["planned_expiration_time"] = pd.to_datetime(loans["planned_expiration_time"])

In [10]:
loans["duration"] = (loans["planned_expiration_time"] - loans["disburse_time"]).dt.days

In [11]:
loans["duration"]

0          53.0
1          96.0
2          37.0
3          34.0
4          57.0
           ... 
1419602    39.0
1419603    39.0
1419604    51.0
1419605    63.0
1419606    61.0
Name: duration, Length: 1419607, dtype: float64

#### 3. Find the lenders that have funded at least twice.

In [12]:
lenders = pd.read_csv("Datasets/lenders.csv")

Utilizzeremo la funzione *.loc* per trovare i lenders che abbiano finanziato almeno due volte.

In [13]:
lenders.describe()

,member_since,loan_purchase_num,num_invited
count,2.349174e+06,1.454893e+06,2.349174e+06
mean,1.352954e+09,1.723265e+01,2.253443e-01
std,9.643617e+07,1.853678e+02,1.691738e+01
min,1.136106e+09,0.000000e+00,0.000000e+00
25%,1.268534e+09,1.000000e+00,0.000000e+00
50%,1.364078e+09,2.000000e+00,0.000000e+00
75%,1.429845e+09,9.000000e+00,0.000000e+00
max,1.515617e+09,8.519000e+04,2.485400e+04


In [14]:
lenders.loc[lenders["loan_purchase_num"] >= 2]["permanent_name"]

1               reena6733
3          andrew86079135
6                rene7585
7              harald2826
11           jennifer4328
                ...      
2349158             rakhi
2349162     james75291930
2349164         carol8279
2349166      eric91401545
2349171         maria2141
Name: permanent_name, Length: 877574, dtype: object

#### 4. For each country, compute how many loans have involved that country as borrowers.

Raggruppiamo per nome e contiamo il numero di loan che sono stati richiesti.

In [15]:
loans.groupby("country_name").count()["loan_id"]

country_name
Afghanistan        2337
Albania            3075
Armenia           13952
Azerbaijan        10172
Belize              218
                  ...  
Vietnam           21839
Virgin Islands        2
Yemen              4206
Zambia             1277
Zimbabwe           5513
Name: loan_id, Length: 96, dtype: int64

#### 5. For each country, compute the overall amount of money borrowed.

Come al punto precedente, ma questa volta sommiamo il *loan_amount*.

In [16]:
loans.groupby("country_name")["loan_amount"].sum()

country_name
Afghanistan        1967950.0
Albania            4307350.0
Armenia           22950475.0
Azerbaijan        14784625.0
Belize              150175.0
                     ...    
Vietnam           24681100.0
Virgin Islands       10000.0
Yemen              3444000.0
Zambia             1978975.0
Zimbabwe           5851875.0
Name: loan_amount, Length: 96, dtype: float64

#### 6. Like the previous point, but expressed as a percentage of the overall amount lent.

Per esprimere come percentuale del totale bisogna prima calcolare il totale con cui poi dividere i vari risultati intermedi.

In [17]:
tot_loan_amount = loans["loan_amount"].sum()
tot_loan_amount

1181437300.0

In [18]:
d = loans.groupby("country_name")["loan_amount"].sum() / tot_loan_amount * 100
d

country_name
Afghanistan       0.166573
Albania           0.364586
Armenia           1.942589
Azerbaijan        1.251410
Belize            0.012711
                    ...   
Vietnam           2.089074
Virgin Islands    0.000846
Yemen             0.291509
Zambia            0.167506
Zimbabwe          0.495318
Name: loan_amount, Length: 96, dtype: float64

Controlliamo il risultato sommando tutte le percentuali.

In [19]:
d.sum()

100.0

#### 7. Like the three previous points, but split for each year (with respect to *disburse time*).

Per calcolare la percentuale per ogni anno prima ricaviamo l'anno.

In [20]:
loans["year"] = loans["disburse_time"].dt.year

Successivamente ricaviamo per ogni anno e per ogni paese il quantitativo di soldi presi in prestito.

In [21]:
loan_country = pd.DataFrame(loans.groupby(["year","country_name"])["loan_amount"].sum())

E adesso calcoliamo i totali di soldi prestati per ogni anno che ci servirà per effettuare la divisione.

In [22]:
amount_per_year = pd.DataFrame(loans.groupby("year")["loan_amount"].sum())

Uniamo le due tabelle e poi calcoliamo la percentuale.

In [23]:
res = pd.merge(loan_country, amount_per_year, on = "year", suffixes = ("", "_tot_year")).reset_index()

In [24]:
res["perc_per_year"] = res["loan_amount"]/res["loan_amount_tot_year"] * 100

In [25]:
res

,year,loan_amount,loan_amount_tot_year,perc_per_year
0,2005.0,1850.0,102850.0,1.798736
1,2005.0,1625.0,102850.0,1.579971
2,2005.0,5000.0,102850.0,4.861449
3,2005.0,5000.0,102850.0,4.861449
4,2005.0,33300.0,102850.0,32.377248
...,...,...,...,...
743,2018.0,14300.0,990025.0,1.444408
744,2018.0,26500.0,990025.0,2.676700
745,2018.0,21750.0,990025.0,2.196914
746,2018.0,8000.0,990025.0,0.808060


Controlliamo che il risultato sia corretto sommando per ogni anno le percentuali.

In [26]:
res.groupby("year")["perc_per_year"].sum()

year
2005.0    100.0
2006.0    100.0
2007.0    100.0
2008.0    100.0
2009.0    100.0
2010.0    100.0
2011.0    100.0
2012.0    100.0
2013.0    100.0
2014.0    100.0
2015.0    100.0
2016.0    100.0
2017.0    100.0
2018.0    100.0
Name: perc_per_year, dtype: float64

#### 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

Uniamo le due tabelle per ottenere per ogni *loan_id* il corrispettivo *lender*. Dato che per ogni *loan_id* ci possono essere più lenders, in tal caso vengono duplicate le colonne.

In [27]:
df1 = pd.merge(loans, loans_lenders, on = "loan_id")
df1 = df1[["loan_id", "loan_amount", "lenders"]]

Per ottenere il numero di lenders per ogni loans usiamo *count()*.

In [28]:
df2 = df1.groupby("loan_id").count()["lenders"]

In [29]:
df2 = pd.DataFrame(df2).rename(columns = {"lenders":"num"}).reset_index()

Uniamo le due tabelle per avere la colonna *num* che conta il numero di lenders.

In [30]:
result = pd.merge(df1, df2, on = "loan_id") 

In [31]:
result[["loan_id", "num", "loan_amount","lenders"]]

,loan_id,num,loan_amount,lenders
0,657307,3,125.0,spencer5657
1,657307,3,125.0,matthew8640
2,657307,3,125.0,larry71496105
3,657259,7,400.0,ltr
4,657259,7,400.0,andrew5306
...,...,...,...,...
28293907,989240,5,175.0,jensdamsgaardvanar
28293908,989240,5,175.0,david47598776
28293909,989240,5,175.0,bernd9221
28293910,989240,5,175.0,valeria98599473


Calcoliamo ora il vero contributo prestato dividento l'importo per il numero di lenders.

In [32]:
result["true_amount"] = result["loan_amount"]/result["num"]

Raggruppiamo ottenendo il risultato

In [33]:
result.groupby("lenders")["true_amount"].sum()

lenders
 000               1672.618411
 00000             1380.693644
 0002              2472.563566
 00mike00            52.631579
 0101craign0101    2623.565117
                      ...     
zzanita              87.500000
zzcyna7269           55.357143
zzinnia              38.000000
zzmcfate           2287.291955
zzrvmf8538          106.417625
Name: true_amount, Length: 1639026, dtype: float64

#### 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

Per prima cosa ottengo la distribuzione dei paesi presenti all'interno del dataset lenders.

In [34]:
# Conto le presenze di ogni paese
d1 = pd.DataFrame(lenders.groupby("country_code")["country_code"].count())

In [35]:
d2 = d1.rename(columns = {"country_code": "num"}).reset_index()

Ora calcolo la percentuale di presenza di ogni paese.

In [36]:
prob = d2["num"] / d2["num"].sum()

Per riempire i *missing values* sostituisco i valori mancanti con valori distribuiti secondo le proporzioni di probabilità ottenute alla riga precedente.

In [37]:
lenders["country_code"].fillna(pd.Series(np.random.choice(d2["country_code"], 
                                                      p=prob, size=len(lenders))), inplace = True)

In [38]:
lenders["country_code"]

0          US
1          US
2          CA
3          PT
4          US
           ..
2349169    GB
2349170    US
2349171    US
2349172    US
2349173    US
Name: country_code, Length: 2349174, dtype: object

Dopo aver corretto i missing values posso calcolare il totale dei soldi prestati.

In [39]:
df1 = pd.merge(loans_lenders, loans, on = "loan_id")

In [40]:
df2 = pd.merge(df1, lenders, left_on = "lenders", right_on = "permanent_name", suffixes = ("_loaners","_lenders"))

Calcolo il totale dei soldi presi in prestito raggruppando per il paese dei *loaners*.

In [41]:
money_borrowed = pd.DataFrame(df2.groupby("country_code_loaners")["loan_amount"].sum()).reset_index()

Calcolo il totale dei soldi prestati raggruppando per il paese dei *lenders*.

In [42]:
money_lent = pd.DataFrame(df2.groupby("country_code_lenders")["loan_amount"].sum()).reset_index()

In [43]:
money = pd.merge(money_borrowed, money_lent, left_on = "country_code_loaners", right_on = "country_code_lenders", suffixes = ("_loaners","_lenders"))
money["difference"] = money["loan_amount_loaners"] - money["loan_amount_lenders"]
money = money.rename(columns = {"country_code_loaners":"country_code"}).drop("country_code_lenders", axis = 1)
money

,country_code,loan_amount_loaners,loan_amount_lenders,difference
0,AF,1966925.0,131750.0,1835175.0
1,AL,4283750.0,28500.0,4255250.0
2,AM,22901100.0,37075.0,22864025.0
3,AZ,14749025.0,7650.0,14741375.0
4,BA,476275.0,97450.0,378825.0
...,...,...,...,...
84,XK,3083025.0,15475.0,3067550.0
85,YE,3440250.0,47875.0,3392375.0
86,ZA,1006525.0,689900.0,316625.0
87,ZM,1975075.0,18950.0,1956125.0


Valori positivi indicano che il paese ha ricevuto più di quanto ha prestato, viceversa significa che il paese ha prestato più soldi di quanto ha ricevuto

#### 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [46]:
country_stats = pd.read_csv("Datasets/country_stats.csv")

Utilizzando la tabella ricavata nel punto precedente la uniamo con la tabella contenente i dati sulla popolazione dei paesi.

In [47]:
country = pd.merge(country_stats, money, on = "country_code")
country["ratio"] = country["difference"]/country["population"]
m = country["ratio"].idxmax()
country.loc[m]

country_name                           Samoa
country_code                              WS
country_code3                            WSM
continent                            Oceania
region                             Polynesia
population                            196440
population_below_poverty_line            NaN
hdi                                    0.702
life_expectancy                          NaN
expected_years_of_schooling              NaN
mean_years_of_schooling                  NaN
gni                                      NaN
kiva_country_name                      Samoa
loan_amount_loaners              1.21832e+07
loan_amount_lenders                      800
difference                       1.21824e+07
ratio                                62.0159
Name: 83, dtype: object

#### 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

La risoluzione è identica a quella del punto precedente.

In [48]:
country["ratio2"] = country["difference"]/country["population_below_poverty_line"]
n = country["ratio2"].idxmax()
country.loc[n]

country_name                            Philippines
country_code                                     PH
country_code3                                   PHL
continent                                      Asia
region                           South-eastern Asia
population                                104918090
population_below_poverty_line                  21.6
hdi                                        0.681799
life_expectancy                               68.34
expected_years_of_schooling                  11.733
mean_years_of_schooling                     9.32687
gni                                         8395.09
kiva_country_name                       Philippines
loan_amount_loaners                     9.06737e+07
loan_amount_lenders                      1.3515e+06
difference                              8.93222e+07
ratio                                      0.851352
ratio2                                  4.13529e+06
Name: 9, dtype: object

#### 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

Importiamo la libreria datetime che ci servirà successivamente.

In [49]:
import datetime

Selezioniamo solo le variabili che ci saranno utili.

In [50]:
df = loans[["loan_id","loan_amount", "disburse_time", "planned_expiration_time"]]

Dividiamo il dataset, in base alla differenza o non tra l'anno di *disburse_time* e l'anno di *planned_expiration_time*.

In [51]:
same_year = df[df["disburse_time"].dt.year == df["planned_expiration_time"].dt.year]
diff_year = df[df["disburse_time"].dt.year != df["planned_expiration_time"].dt.year]

Calcoliamo il risultato per il dataset *same_year*, e successivamente sommeremo questo risultato ai risultati ottenuti dall'altra tabella.

In [52]:
same_year["year"] = same_year["disburse_time"].dt.year
same = pd.DataFrame(same_year.groupby("year")["loan_amount"].sum())

C:\Users\Marco\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Per lavorare con l'altro dataset inizialmente eliminiamo le righe che presentano valori mancanti oppure se *planned_expiration_time* è precedente a *disburse_time*.

In [53]:
diff_year = diff_year.loc[(diff_year["planned_expiration_time"].isnull() == False) | (diff_year["disburse_time"].isnull() == False)]
diff_year = diff_year.loc[diff_year["planned_expiration_time"] > diff_year["disburse_time"]]

Adesso ripetiamo le righe per il numero di anni che intercorrono, ovviamente se solo un anno è passato la riga verrà ripetuta una sola volta.

In [54]:
diff_year = diff_year.loc[diff_year.index.repeat(diff_year["planned_expiration_time"].dt.year - diff_year["disburse_time"].dt.year + 1)]

Creiamo ora tre nuove colonne, una rappresenta un conteggio della ripetizione delle righe (*count_year*), l'altra invece è un conteggio in termini di anni a partire dall'anno di *disburse_time* fino all'anno di *planned_expiration_time* (*count*). Infine l'ultima ha come valore la fine dell'anno di disburse_time (*end_year*).

In [55]:
diff_year["count"] = diff_year.groupby("loan_id").cumcount()
diff_year["count_year"] = pd.to_datetime(diff_year.groupby("loan_id").cumcount() + diff_year["disburse_time"].dt.year, format = "%Y")
diff_year["end_year"] = diff_year["disburse_time"].dt.year.apply(lambda x: datetime.datetime(x, 12, 31))

Dato che per i passaggi successivi la timezone potrebbe creare qualche problema, la eliminiamo dalle date. Inoltre resettiamo l'indice per lo stesso motivo precedente.

In [56]:
diff_year["planned_expiration_time"] = diff_year["planned_expiration_time"].dt.tz_localize(None)
diff_year["disburse_time"] = diff_year["disburse_time"].dt.tz_localize(None)
diff_year = diff_year.reset_index()

Definiamo ora la colonna *useful_days*, alla quale, nel caso il conteggio in termini di anni sia uguale al *disburse_time* assegnamo il valore della differenza (in giorni) tra la fine dell'anno e il *disburse_time*. Invece se è diverso assegna la differenza (sempre in giorni) tra *planned_expiration_time* e il primo giorno dell'anno di *count_year*.

In [57]:
diff_year.loc[diff_year["count_year"].dt.year == diff_year["disburse_time"].dt.year, "useful_days"] = (diff_year["end_year"] - diff_year["disburse_time"]).dt.days
diff_year.loc[diff_year["count_year"].dt.year != diff_year["disburse_time"].dt.year, "useful_days"] = (diff_year["planned_expiration_time"] - diff_year["count_year"]).dt.days

Ovviamente sorgono problemi, poichè tutte le righe duplicate che presentano un conteggio diverso da 0 e diverso dall'ultimo numero disponibile presentano la differenza in giorni tra *planned_expiration_time* e l'inizio ma **di un anno diverso**. Per correggere assegnamo a tutti i valori superiori a 365 (i giorni in un anno) il valore di 365. Se l'anno è bisestile invece assegnamo il valore di 366.

In [58]:
diff_year.loc[(diff_year["useful_days"] > 365) & (diff_year["count_year"].dt.is_leap_year == True), "useful_days"] = 366
diff_year.loc[(diff_year["useful_days"] > 365) & (diff_year["count_year"].dt.is_leap_year == False), "useful_days"] = 365

Dopo che abbiamo ottenuto i risultati possiamo finalmente procedere a calcolare il *loan_amount* per anno. Per prima cosa raggruppiamo per *loan_id* e sommiamo gli *useful_days*, così da ottenere il totale dei giorni per ogni loan.
Successivamente aggiungiamo l'informazione appena ottenuta al nostro dataset precedente *diff_year*, e calcoliamo la proporzione di *loan_amount* rispetto ai giorni.

In [59]:
days = pd.DataFrame(diff_year.groupby("loan_id")["useful_days"].sum())
result = pd.merge(diff_year, days, on = "loan_id", suffixes = ("","_tot"))
result["true_amount"] = result["loan_amount"]*result["useful_days"] / result["useful_days_tot"]
result

,index,loan_id,loan_amount,disburse_time,planned_expiration_time,count,count_year,end_year,useful_days,useful_days_tot,true_amount
0,0,657307,125.0,2013-12-22 08:00:00,2014-02-14 03:30:06,0,2013-01-01,2013-12-31,8.0,52.0,19.230769
1,0,657307,125.0,2013-12-22 08:00:00,2014-02-14 03:30:06,1,2014-01-01,2013-12-31,44.0,52.0,105.769231
2,1,657259,400.0,2013-12-20 08:00:00,2014-03-26 22:25:07,0,2013-01-01,2013-12-31,10.0,94.0,42.553191
3,1,657259,400.0,2013-12-20 08:00:00,2014-03-26 22:25:07,1,2014-01-01,2013-12-31,84.0,94.0,357.446809
4,4,656933,425.0,2013-12-17 08:00:00,2014-02-13 06:10:02,0,2013-01-01,2013-12-31,13.0,56.0,98.660714
...,...,...,...,...,...,...,...,...,...,...,...
302198,1419604,989109,2425.0,2015-11-13 08:00:00,2016-01-03 22:20:04,1,2016-01-01,2015-12-31,2.0,49.0,98.979592
302199,1419605,989143,100.0,2015-11-03 08:00:00,2016-01-05 08:50:02,0,2015-01-01,2015-12-31,57.0,61.0,93.442623
302200,1419605,989143,100.0,2015-11-03 08:00:00,2016-01-05 08:50:02,1,2016-01-01,2015-12-31,4.0,61.0,6.557377
302201,1419606,989240,175.0,2015-11-03 08:00:00,2016-01-03 20:50:06,0,2015-01-01,2015-12-31,57.0,59.0,169.067797


Adesso possiamo raggruppare per anno (*count_year* poichè è l'unica varaibile che tiene conto dei vari anni) e sommare il *true_amount*.

In [60]:
diff = pd.DataFrame(result.groupby("count_year")["true_amount"].sum()).reset_index()
diff["count_year"] = diff["count_year"].dt.year

Infine sommiamo i risultati con la tabella *same* calcolata precedentemente.

In [61]:
final = pd.merge(diff,same, left_on = "count_year", right_on="year")
final["amount_per_year"] = final["true_amount"] + final["loan_amount"]

In [62]:
final[["count_year","amount_per_year"]]

,count_year,amount_per_year
0,2012,1.132958e+08
1,2013,1.249480e+08
2,2014,1.531481e+08
3,2015,1.597593e+08
4,2016,1.589647e+08
5,2017,1.719490e+08
6,2018,5.633502e+06
